In [1]:
import tensorflow as tf

import pkg_resources as pkg_rs
tensorflow_version = pkg_rs.get_distribution("tensorflow").version
print tensorflow_version

0.9.0rc0


In [2]:
import cPickle as pickle
import numpy as np

In [3]:
flags = tf.app.flags
FLAGS = flags.FLAGS

flags.DEFINE_string('summaries_dir', '../tensorboard1', 'Summaries directory')

# launch tensorboard with
# tensorboard --logdir='/Users/peterhirt/datascience/ild-cnn/tensorboard1'

if the tensorboard data directory exists, delete the content and create an empty directory

In [4]:
if tf.gfile.Exists(FLAGS.summaries_dir):
    tf.gfile.DeleteRecursively(FLAGS.summaries_dir)
tf.gfile.MakeDirs(FLAGS.summaries_dir)

# load ILD_CNN dataset

both training set and validation set is loaded, normalised and the labels hot-encoded.

In [5]:
def load_data():

    # load the dataset as X_train and as a copy the X_val
    # the labels are loaded as one-hot encoded data
    
    X_train = pickle.load( open( "../pickle/X_train.pkl", "rb" ) )
    y_train = pickle.load( open( "../pickle/y_train_hot.pkl", "rb" ) )
    X_val = pickle.load( open( "../pickle/X_val.pkl", "rb" ) )
    y_val = pickle.load( open( "../pickle/y_val_hot.pkl", "rb" ) )
    
    X_test = pickle.load( open( "../pickle/X_test.pkl", "rb" ) )
    y_test = pickle.load( open( "../pickle/y_test_hot.pkl", "rb" ) )


    

    # adding a singleton dimension and rescale to [0,1]
    X_train = np.asarray(np.expand_dims(X_train, axis=3))/float(255)
    X_val = np.asarray(np.expand_dims(X_val,axis=3))/float(255)
    X_test = np.asarray(np.expand_dims(X_test,axis=3))/float(255)
    
    X_train = np.float32(X_train)
    X_val = np.float32(X_val)
    X_test = np.float32(X_test)
    
    
    

    

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

In [6]:
# load the dataset
# the labels are one-hot-encoded

(X_train, y_train), (X_val, y_val), (X_test, y_test) = load_data()


(12500, 32, 32, 1)
float32
(12500, 6)
float32
(6250, 32, 32, 1)
(6250, 6)


In [30]:
# shape and type of the training set
print(X_train.shape)
print(y_train.shape)
print 'type of training set: ', X_train.dtype 

print 'shape of test set'
print(X_test.shape)
print(y_test.shape)


(12500, 32, 32, 1)
(12500, 6)
type of training set:  float32
shape of test set
(6250, 32, 32, 1)
(6250, 6)


# model definitions
    batch size : 128
    interations : 10000

In [25]:
# Parameters
learning_rate = 0.001
training_iters = 10000
batch_size = 128
display_step = 10

In [8]:
# Network Parameters

dropout = 0.5 # Dropout, probability to keep units
keep_prob = tf.placeholder(tf.float32)

# tf Graph input
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 32, 32, 1], name='x-input')
    y = tf.placeholder(tf.float32, [None, 6], name='y-input')
    

# dropout (keep probability)
 

In [9]:
def next_batch(batch_size):
    
    n_samples = len(X_train)
    indices = np.random.choice(n_samples, batch_size)
    batch_x = X_train[indices]
    batch_y = y_train[indices]
    
    return batch_x, batch_y

In [16]:
def batch_flatten(x):
    # Turn a n-D tensor into a 2D tensor where the first dimension is conserved.
    
    x = tf.reshape(x, [-1, np.prod(x.get_shape()[1:].as_list())])
    return x

In [10]:
def variable_summaries(var, name):
    
    mean = tf.reduce_mean(var)
    tf.scalar_summary('mean/' + name, mean)
    stddev = tf.sqrt(tf.reduce_sum(tf.square(var - mean)))
    
    tf.scalar_summary('sttdev/' + name, stddev)
    tf.scalar_summary('max/' + name, tf.reduce_max(var))
    tf.scalar_summary('min/' + name, tf.reduce_min(var))
    tf.histogram_summary(name, var)
    
def conv2d(x, W, b, convLayer, strides=1):
    with tf.name_scope(convLayer):
        # Conv2D wrapper, with bias and relu activation
        x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME', name=convLayer)
        x = tf.nn.bias_add(x, b)
        #variable_summaries(W, convLayer + '/weights')
        #variable_summaries(b, convLayer + '/biases')
        return tf.nn.relu(x)


def maxpool2d(x, k):
    with tf.name_scope('AvgPooling'):
        # MaxPool2D wrapper
        return tf.nn.avg_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

In [11]:
# Store layers weight & bias
with tf.name_scope('weights'):
    weights = {
        # 2x2 conv, 1 input, 16 outputs
        'wc1': tf.Variable(tf.random_normal([2, 2, 1, 16])),
        # 2x2 conv, 16 inputs, 36 outputs
        'wc2': tf.Variable(tf.random_normal([2, 2, 16, 36])),
        # 2x2 conv, 36 inputs, 64 outputs
        'wc3': tf.Variable(tf.random_normal([2, 2, 36, 64])),
        # 2x2 conv, 64 inputs, 100 outputs
        'wc4': tf.Variable(tf.random_normal([2, 2, 64, 100])),
        # 2x2 conv, 100 inputs, 144 outputs
        'wc5': tf.Variable(tf.random_normal([2, 2, 100, 144])),
        
        # fully connected, 144 inputs, 864 outputs
        'wd1': tf.Variable(tf.random_normal([144, 864])),
        # fully connected, 144 inputs, 864 outputs
        'wd2': tf.Variable(tf.random_normal([864, 288])),
        # 1024 inputs, 10 outputs (class prediction)
        'out': tf.Variable(tf.random_normal([288, 6]))
        }

with tf.name_scope('bias'):
    biases = {
        'bc1': tf.Variable(tf.random_normal([16])),
        'bc2': tf.Variable(tf.random_normal([36])),
        'bc3': tf.Variable(tf.random_normal([64])),
        'bc4': tf.Variable(tf.random_normal([100])),
        'bc5': tf.Variable(tf.random_normal([144])),
    
        'bd1': tf.Variable(tf.random_normal([864])),
        'bd2': tf.Variable(tf.random_normal([288])),
        'out': tf.Variable(tf.random_normal([6]))
        }

In [21]:
# Create model

def conv_net(x, weights, biases):
    
    # Convolution Layer 1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'], 'convolution2D_1')
    
    # Convolution Layer 2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'], 'convolution2D_2')
    
    # Convolution Layer 3
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'], 'convolution2D_3')
    
    # Convolution Layer 4
    conv4 = conv2d(conv3, weights['wc4'], biases['bc4'], 'convolution2D_4')
    
    # Convolution Layer 5
    conv5 = conv2d(conv4, weights['wc5'], biases['bc5'], 'convolution2D_5')
    
    pool1 = maxpool2d(conv5, k=32)
    
    # must do flattening and dropout first
    #pool1_reshaped = tf.squeeze(pool1, [1, 2])
    pool1_reshaped = batch_flatten(pool1)
    pool1_dropout = tf.nn.dropout(pool1_reshaped, keep_prob)
    
    with tf.name_scope('dense1'):
        
        # pool1 should be 144 rank 1 tensor, weights should be 864x144, bias should be 864
        h_fc1 = tf.nn.relu(tf.matmul(pool1_dropout, weights['wd1']) + biases['bd1'])
        
        # again dropout op needed
        
    with tf.name_scope('dense2'):
        # dense
        # relu
        h_fc2 = tf.nn.relu(tf.matmul(h_fc1, weights['wd2']) + biases['bd2'])
        # dropout
    
    with tf.name_scope('dense3'):
        # dense with softmax
        y_conv=tf.nn.softmax(tf.matmul(h_fc2, weights['out']) + biases['out'])


    
    return y_conv    

In [22]:
# Construct model
pred = conv_net(x, weights, biases)


# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [26]:
# Initializing the variables

init = tf.initialize_all_variables()

In [27]:
# Launch the graph

with tf.Session() as sess:
    merged = tf.merge_all_summaries()
    writer = tf.train.SummaryWriter(FLAGS.summaries_dir, sess.graph)
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        
        # prepare the next batch
        batch_x, batch_y = next_batch(batch_size)
        
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
        
        run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata = tf.RunMetadata()
        
        sess.run(pred, 
                 feed_dict={x: batch_x, y: batch_y, keep_prob: dropout},
                 options=run_options,
                 run_metadata=run_metadata)
        
        
        
        if step % display_step == 0:
            
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
            
        step += 1
    print "Optimization Finished!"

    # Calculate accuracy for 256 mnist test images
    print "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: X_test[:256], y: y_test[:256], keep_prob: 1.})

Iter 1280, Minibatch Loss= 1.918591, Training Accuracy= 0.12500
Iter 2560, Minibatch Loss= 1.895154, Training Accuracy= 0.14844
Iter 3840, Minibatch Loss= 1.902966, Training Accuracy= 0.14062
Iter 5120, Minibatch Loss= 1.809216, Training Accuracy= 0.23438
Iter 6400, Minibatch Loss= 1.918591, Training Accuracy= 0.12500
Iter 7680, Minibatch Loss= 1.918591, Training Accuracy= 0.12500
Iter 8960, Minibatch Loss= 1.910779, Training Accuracy= 0.13281
Optimization Finished!
Testing Accuracy: 0.171875
